<a href="https://colab.research.google.com/github/dainilliniad/small311qlab/blob/main/%D0%BA%D1%83%D1%80%D1%81%D0%BE%D0%B2%D0%B0%D1%8F_%D0%B3%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#выгрузка необходимых библиотек
import math
import torch
import matplotlib.pyplot as plt
import random
import numpy as np
from torch import unsqueeze as unsQ
import time
import tqdm
from tqdm import tqdm

In [ ]:
#создание оригинала изображения
def onePatOfObject(x, y, pmax = 0.9): #degree  радианах, идеальность от 0 до 1, pmin неидеальности диапазоны
    p=torch.rand(x, y)*pmax
    tetta=torch.rand(x,y)*np.pi/2
    return torch.tensor(p),torch.tensor(tetta)


#создание одного шаблона освещения
def createRandomPattern(width, height, seed = np.nan, device = 'cpu'):
  if not(np.isnan(seed)) : torch.manual_seed(seed)
  return torch.randint(low = 0, high = 2, size = (width, height), device = device,dtype = torch.float)


#одного измерение сигнала после объекта
def createObjectSignal(pattern,objectProfile, device = 'cpu'):
  return unsQ(torch.sum(torch.mul(objectProfile, pattern)),0)


# создание последовательности симулированных данных для объекта

def createEmulData(numbOfMeasur, objectProfile, device = 'cpu'):
  width = objectProfile.size()[0]
  height = objectProfile.size()[1]
  patterns = unsQ(createRandomPattern(width, height, device = device),0)
  objData = createObjectSignal(patterns[0],objectProfile, device = device)
  for num in range(1,numbOfMeasur):
    localPat = createRandomPattern(width, height, device = device)
    objData = torch.cat((objData,createObjectSignal(localPat,objectProfile, device = device)),0)
    patterns = torch.cat((patterns,unsQ(localPat,0)),0)
  return patterns, objData

def crossCorrelation(patterns, objData, device = 'cpu'):
  localLen = objData.shape[0]
  correlation = torch.zeros_like(patterns[0])
  for i in range(localLen):
    correlation = correlation + patterns[i]*objData[i]/localLen
  return (correlation - torch.mean(patterns,0)*torch.mean(objData))*4

In [ ]:
#cpu or gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [ ]:
#задание настраиваемых параметров:
x, y = 7, 7 # размеры картинки
pmax1=0.9 #максимальное значение величины анизотропии p
num_frames=10000 #число шаблонов освещения для приближения корреляционой функции
num_objs=400 #число объектов

In [ ]:
print(x,y)

1 2


In [ ]:
##создание выборки##
pitog=[]
tetitog=[]
cor1=[]
cor2=[]
cor3=[]
for i in tqdm(range(num_objs)):
  obj=onePatOfObject(x, y, pmax = pmax1)
  p, tet=obj[0], obj[1]
  T0=torch.cos(tet)*torch.cos(tet)+p*torch.sin(tet)*torch.sin(tet)
  T22=1/2*(1+p*p-(1-p*p)*torch.sin(2*tet))
  T45=torch.sin(tet)*torch.sin(tet)+p*torch.cos(tet)*torch.cos(tet)
  pat_test,objData_test = createEmulData(num_frames,T0,device = device)
  correlation1 = crossCorrelation(pat_test,objData_test,device = device)
  pat_test,objData_test = createEmulData(num_frames,T22,device = device)
  correlation2 = crossCorrelation(pat_test,objData_test,device = device)
  pat_test,objData_test = createEmulData(num_frames,T45,device = device)
  correlation3 = crossCorrelation(pat_test,objData_test,device = device)


  for j in range(7):
    for k in range(7):
      pitog.append(float(p[j][k]))
      tetitog.append(float(tet[j][k]))
      cor1.append(float(correlation1[j][k]))
      cor2.append(float(correlation2[j][k]))
      cor3.append(float(correlation3[j][k]))
np.save('pitog', pitog)
np.save('tetitog', tetitog)
np.save('cor1', cor1)
np.save('cor2', cor2)
np.save('cor3', cor3)

In [ ]:
pkras=[[0.0, 0.166, 0.332, 0.498, 0.664, 0.83, 0.996],
       [0.166, 0.332, 0.498, 0.664, 0.83, 0.996, 0.0],
       [0.332, 0.498, 0.664, 0.83, 0.996, 0.0, 0.166],
       [0.498, 0.664, 0.83, 0.996, 0.0, 0.166, 0.332],
       [0.664, 0.83, 0.996, 0.0, 0.166, 0.332, 0.498],
       [0.83, 0.996, 0.0, 0.166, 0.332, 0.498, 0.664],
       [0.996, 0.0, 0.166, 0.332, 0.498, 0.664, 0.83]]

tetkras=[[0.0, 0.166, 0.332, 0.498, 0.332, 0.166, 0.0],
        [0.166, 0.332, 0.498, 0.664, 0.498, 0.332, 0.166],
        [0.332, 0.498, 0.664, 0.83,  0.664, 0.498, 0.332],
        [0.498, 0.664, 0.83, 0.996, 0.83, 0.664, 0.498],
        [0.332, 0.498, 0.664, 0.83,  0.664, 0.498, 0.332],
        [0.166, 0.332, 0.498, 0.664, 0.498, 0.332, 0.166],
        [0.0, 0.166, 0.332, 0.498, 0.332, 0.166, 0.0]]


In [ ]:
p, tet=torch.tensor(pkras), torch.tensor(tetkras)*1.57
print(p)
T0=torch.cos(tet)*torch.cos(tet)+p*torch.sin(tet)*torch.sin(tet)
T22=1/2*(1+p*p-(1-p*p)*torch.sin(2*tet))
T45=torch.sin(tet)*torch.sin(tet)+p*torch.cos(tet)*torch.cos(tet)
pat_test,objData_test = createEmulData(10000,T0,device = device)
correlation1 = crossCorrelation(pat_test,objData_test,device = device)
pat_test,objData_test = createEmulData(10000,T22,device = device)
correlation2 = crossCorrelation(pat_test,objData_test,device = device)
pat_test,objData_test = createEmulData(10000,T45,device = device)
correlation3 = crossCorrelation(pat_test,objData_test,device = device)

p_guess=correlation1+correlation3-1

p_guess[p_guess>1]=1
p_guess[p_guess<0]=0
p_guess=torch.sqrt(p_guess)

plt.imshow(p_guess.to('cpu'))
plt.colorbar()
plt.show()
